# ML Pipeline Preparation

### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [4]:
# import libraries
import nltk
nltk.download(['punkt', 'wordnet', 'stopwords'])

import re
import numpy as np
import pandas as pd

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

from sqlalchemy import create_engine


[nltk_data] Downloading package punkt to
[nltk_data]     /Users/oladipooluwadamilola/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/oladipooluwadamilola/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/oladipooluwadamilola/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [5]:
# load data from sqlite database
engine = create_engine('sqlite:////home/workspace/DisasterResponse.db')
df = pd.read_sql_table('cleaned_df', con=engine)
df.info()

OperationalError: (sqlite3.OperationalError) unable to open database file
(Background on this error at: http://sqlalche.me/e/14/e3q8)

In [ ]:
df.head(1)

In [ ]:
#check for class imbalance for all the columns

In [4]:
df.message.values

array(['Weather update - a cold front from Cuba that could pass over Haiti',
       'Is the Hurricane over or is it not over',
       'Looking for someone but no name', ...,
       "Proshika, operating in Cox's Bazar municipality and 5 other unions, Ramu and Chokoria, assessment, 5 kg rice, 1,5 kg lentils to 700 families.",
       'Some 2,000 women protesting against the conduct of the elections were teargassed as they tried to converge on the local electoral commission offices in the southern oil city of Port Harcourt.',
       'A radical shift in thinking came about as a result of this meeting, recognizing that HIV/AIDS is at the core of the humanitarian crisis and identifying the crisis itself as a function of the HIV/AIDS pandemic.'], dtype=object)

### 2. Write a tokenization function to process your text data

In [2]:
def tokenize(text):
    '''
    input: 
        raw texts and returns
    output: 
        clean_tokens- cleaned list of words in the text after tokenizing and lemmatizing
    '''
    
    text = re.sub(r'[^A-Za-z0-9]'," ",text.lower()) #NORMALIZE- make lowercase, remove character not alphanumeric

    tokens = word_tokenize(text) #TOKENIZE - returns a list of words(toks) in the input text
    
    lemmatizer = WordNetLemmatizer() #initialize lemmatizer
    
    #stopwords removal and strip of whitespace
    stop_tokens = [word.strip() for word in tokens if word not in stopwords.words("english")]
    
    clean_tokens = [lemmatizer.lemmatize(tok) for tok in stop_tokens] # LEMMATIZING
    
    return clean_tokens

    
    
    
    
    
    
    

""""
#stopwords removal and strip of whitespace
stop_tokens = []
for word in tokens:
    if word not in stopwords.words("english"):
        stop_tokens.append(word.strip())

# LEMMATIZING    
clean_tokens = []
for tok in stop_tokens:
    clean_tok = lemmatizer.lemmatize(tok)
    clean_tokens.append(clean_tok)

return clean_tokens
"""

### 3. Build a machine learning pipeline
Build a ml classifier pipeline for this multi-output classificatn that will use the 'message' column as input to predict classificatns for 36 other categories. 
 Check the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) for more info on predicting multiple target variables.

In [6]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer,TfidfTransformer

from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,f1_score,precision_recall_curve,accuracy_score

from sklearn.model_selection import GridSearchCV

In [7]:
#FEATURE EXTRACTION - bag of words and tfidf. use svc clasifier alg
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf',TfidfTransformer()),
    ('multi_clf',MultiOutputClassifier(OneVsRestClassifier(LinearSVC())))
])
 

In [8]:
# Split the dataset into features and labels sets
# features from the 4th col to 40th col
X = df.message.values
Y = df.iloc[:, 4:].values


print(X.shape)
print(Y.shape)

(26215,)
(26215, 36)


### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, Y)

In [10]:
#train the classifier pipeline
pipeline.fit(X_train, y_train) 

#predict on test feature
y_pred = pipeline.predict(X_test)



/opt/conda/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 0 is present in all training examples.
  str(classes[c]))


### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. Use sklearn's `classification_report` on each column.

In [11]:
category_names=['related', 'request', 'offer', 'aid_related', 'medical_help', 'medical_products', 
                   'search_and_rescue', 'security',
                   'military', 'child_alone', 'water', 'food', 
                   'shelter', 'clothing', 'money', 'missing_people', 'refugees', 'death',
                   'other_aid', 'infrastructure_related',
                   'transport', 'buildings', 'electricity', 'tools', 'hospitals', 'shops',
                   'aid_centers', 'other_infrastructure', 
                   'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold',
                   'other_weather', 'direct_report']

In [12]:
print(classification_report(y_test, y_pred, target_names=category_names))

                        precision    recall  f1-score   support

               related       0.87      0.92      0.89      5039
               request       0.72      0.57      0.63      1116
                 offer       0.00      0.00      0.00        30
           aid_related       0.72      0.70      0.71      2703
          medical_help       0.60      0.30      0.40       514
      medical_products       0.62      0.32      0.42       325
     search_and_rescue       0.53      0.12      0.19       156
              security       0.38      0.04      0.08       116
              military       0.63      0.32      0.42       207
           child_alone       0.00      0.00      0.00         0
                 water       0.76      0.59      0.66       418
                  food       0.82      0.68      0.74       744
               shelter       0.78      0.57      0.66       557
              clothing       0.77      0.32      0.45       112
                 money       0.51      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [13]:
#ACCURACY
for i in range(y_test.shape[1]):
    print('{} accuracy: {}'.format(category_names[i],accuracy_score(y_test[:,i],y_pred[:,i])))
    
#accuracy not a good metric for imbalanced data

# #For confusion matrix
# def display_results(cv, y_test, y_pred):
#     labels = np.unique(y_pred)
#     confusion_mat = confusion_matrix(y_test, y_pred, labels=labels)
#     accuracy = (y_pred == y_test).mean()

#     print("Labels:", labels)
#     print("Confusion Matrix:\n", confusion_mat)
#     print("Accuracy:", accuracy)
#     print("\nBest Parameters:", cv.best_params_)


related accuracy: 0.8321635642355814
request accuracy: 0.8884650595056454
offer accuracy: 0.9954226426609704
aid_related accuracy: 0.7607567897467196
medical_help accuracy: 0.9296612755569118
medical_products accuracy: 0.9566676838571865
search_and_rescue accuracy: 0.9765028989929814
security accuracy: 0.9818431492218492
military accuracy: 0.9725358559658224
child_alone accuracy: 1.0
water accuracy: 0.9618553555080867
food accuracy: 0.9465974977113213
shelter accuracy: 0.9501068050045773
clothing accuracy: 0.9867256637168141
money accuracy: 0.9800122062862374
missing_people accuracy: 0.9871833994507171
refugees accuracy: 0.9688739700945987
death accuracy: 0.9694842844064693
other_aid accuracy: 0.8617638083613061
infrastructure_related accuracy: 0.9311870613365884
transport accuracy: 0.9574305767470247
buildings accuracy: 0.9615501983521514
electricity accuracy: 0.9801647848642051
tools accuracy: 0.9948123283490998
hospitals accuracy: 0.9891669209642966
shops accuracy: 0.994812328349099

### 6. Improve your model
Use grid search to find better parameters. 

In [14]:
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x7f6680acaf28>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('multi_clf',
   MultiOutputClassifier(estimator=OneVsRestClassifier(estimator=LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
        intercept_scaling=1, loss='squared_hinge', max_iter=1000,
        multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
        verbose=0),
             n_jobs=1),
              n_jobs=1))],
 'vect': CountVectorizer(analyzer='word', binary=False, decode_error='strict',
   

In [15]:

parameters = {'vect__binary': [False,True],
              'tfidf__norm': ['l1','l2'],
             'multi_clf__estimator__estimator__C': [3.0]
             }



cv = GridSearchCV(pipeline,param_grid=parameters)

cv.fit(X_train, y_train)
y_predict = cv.predict(X_test)

/opt/conda/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 0 is present in all training examples.
  str(classes[c]))
/opt/conda/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 0 is present in all training examples.
  str(classes[c]))
/opt/conda/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 0 is present in all training examples.
  str(classes[c]))
/opt/conda/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 0 is present in all training examples.
  str(classes[c]))
/opt/conda/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 0 is present in all training examples.
  str(classes[c]))
/opt/conda/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 0 is present in all training examples.
  str(classes[c]))
/opt/conda/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 0 is present in all training example

### 7. Test your tuned model 

Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [16]:
print(classification_report(y_test, y_pred, target_names=category_names))

                        precision    recall  f1-score   support

               related       0.85      0.95      0.90      5039
               request       0.75      0.53      0.63      1116
                 offer       0.00      0.00      0.00        30
           aid_related       0.72      0.67      0.69      2703
          medical_help       0.59      0.19      0.28       514
      medical_products       0.69      0.18      0.29       325
     search_and_rescue       0.62      0.06      0.12       156
              security       0.00      0.00      0.00       116
              military       0.61      0.17      0.27       207
           child_alone       0.00      0.00      0.00         0
                 water       0.75      0.42      0.54       418
                  food       0.81      0.51      0.62       744
               shelter       0.75      0.41      0.53       557
              clothing       0.77      0.21      0.34       112
                 money       0.46      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


### 8. Try improving your model further. 
* try other machine learning algorithms
* add other features  besides the TF-IDF

### 9. Export model as a pickle file

In [21]:
import joblib
filepath = 'classifier.pkl'
joblib.dump("multi_clf", filepath)


['classifier.pkl']

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.